In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import liana as li
import decoupler as dc
import os

In [2]:
os.chdir("/Users/ricardoramirez/Dropbox/PhD/Research/MOFAcell")

In [3]:
adata = li.testing.datasets.kang_2018()

100%|███████████████████████████████████████| 36.6M/36.6M [00:41<00:00, 915kB/s]


In [14]:
adata.obs

,nCount_RNA,nFeature_RNA,tsne1,tsne2,condition,cluster,cell_type,patient,nCount_SCT,nFeature_SCT,integrated_snn_res.0.4,seurat_clusters,sample,cell_abbr
index,,,,,,,,,,,,,,
AAACATACATTTCC-1,3017.0,877,-27.640373,14.966629,ctrl,9,CD14+ Monocytes,patient_1016,1704.0,711,1,1,ctrl&1016,CD14
AAACATACCAGAAA-1,2481.0,713,-27.493646,28.924885,ctrl,9,CD14+ Monocytes,patient_1256,1614.0,662,1,1,ctrl&1256,CD14
AAACATACCATGCA-1,703.0,337,-10.468194,-5.984389,ctrl,3,CD4 T cells,patient_1488,908.0,337,6,6,ctrl&1488,CD4T
AAACATACCTCGCT-1,3420.0,850,-24.367997,20.429285,ctrl,9,CD14+ Monocytes,patient_1256,1738.0,653,1,1,ctrl&1256,CD14
AAACATACCTGGTA-1,3158.0,1111,27.952170,24.159738,ctrl,4,Dendritic cells,patient_1039,1857.0,928,12,12,ctrl&1039,DCs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGCATGCCTGAA-2,1033.0,468,18.268321,1.058202,stim,6,CD4 T cells,patient_1244,1128.0,468,2,2,stim&1244,CD4T
TTTGCATGCCTGTC-2,2116.0,819,-11.563067,2.574095,stim,4,B cells,patient_1256,1669.0,799,3,3,stim&1256,B
TTTGCATGCTAAGC-2,1522.0,523,25.142392,6.603815,stim,6,CD4 T cells,patient_107,1422.0,523,0,0,stim&107,CD4T


In [4]:
adata.obs[["cell_type", "cell_abbr"]].drop_duplicates()

,cell_type,cell_abbr
index,,
AAACATACATTTCC-1,CD14+ Monocytes,CD14
AAACATACCATGCA-1,CD4 T cells,CD4T
AAACATACCTGGTA-1,Dendritic cells,DCs
AAACATACGTGTAC-1,NK cells,NK
AAACATTGAGTGTC-1,CD8 T cells,CD8T
AAACATTGCTCGAA-1,B cells,B
AAACGCTGGGTAGG-1,FCGR3A+ Monocytes,FGR3
AAACTTGAAAAACG-1,Megakaryocytes,Mega


In [11]:
#Patient metadata for the sake of completeness
meta_data = adata.obs[["sample", "condition"]].drop_duplicates()
meta_data.reset_index(drop=True,inplace=True)

In [12]:
meta_data

,sample,condition
0,ctrl&1016,ctrl
1,ctrl&1256,ctrl
2,ctrl&1488,ctrl
3,ctrl&1039,ctrl
4,ctrl&1015,ctrl
5,ctrl&107,ctrl
6,ctrl&1244,ctrl
7,ctrl&101,ctrl
8,stim&101,stim
9,stim&1488,stim


In [13]:
meta_data.to_csv("./data_lupus/meta_data.csv")

In [15]:
#Sample cell-type
adata.layers['counts'] = adata.X
padata = dc.get_pseudobulk(adata, sample_col='sample', groups_col='cell_abbr', layer='counts', min_prop=0, min_smpls=0)

In [16]:
padata.var.index.values

array(['7SK.2', 'A1BG', 'A1BG-AS1', ..., 'hsa-mir-1199', 'hsa-mir-335',
       'hsa-mir-8072'], dtype=object)

In [17]:
pb_dat = pd.DataFrame(padata.X)
pb_dat.columns = padata.var.index.values
pb_dat.index = padata.obs.index.values

In [18]:
pb_dat

,7SK.2,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A4GALT,AAAS,AACS,AAED1,...,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,hsa-mir-1199,hsa-mir-335,hsa-mir-8072
ctrl&101_B,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,6.0,...,0.0,0.0,0.0,0.0,5.0,2.0,0.0,0.0,0.0,3.0
ctrl&1015_B,2.0,16.0,3.0,1.0,0.0,0.0,1.0,9.0,1.0,22.0,...,2.0,0.0,3.0,1.0,5.0,5.0,5.0,0.0,0.0,4.0
ctrl&1016_B,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,...,0.0,0.0,1.0,0.0,4.0,4.0,2.0,0.0,0.0,3.0
ctrl&1039_B,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
ctrl&107_B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
stim&1039_NK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
stim&107_NK,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
stim&1244_NK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
stim&1256_NK,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,...,0.0,0.0,0.0,3.0,5.0,0.0,0.0,1.0,0.0,0.0


In [19]:
pb_dat.to_csv("./data_lupus/pb_data.csv")

In [24]:
#Finally coldata of the samples
cell_type_numbers = adata.obs.groupby(["sample", "cell_abbr"])["cell_abbr"].count()
cell_type_numbers = cell_type_numbers.to_frame().rename({"cell_abbr":"counts"}, axis=1).reset_index()

In [25]:
cell_type_numbers

,sample,cell_abbr,counts
0,ctrl&101,CD4T,353
1,ctrl&101,CD14,207
2,ctrl&101,B,106
3,ctrl&101,NK,65
4,ctrl&101,CD8T,29
...,...,...,...
123,stim&1488,NK,149
124,stim&1488,CD8T,42
125,stim&1488,FGR3,90
126,stim&1488,DCs,57


In [26]:
cell_type_numbers.to_csv("./data_lupus/cell_counts.csv")